In [ ]:
import sym_sinekan
import netket as nk
import numpy as np
from netket.operator.spin import sigmax, sigmay, sigmaz
from netket.exact import lanczos_ed
import optax

In [ ]:
def Lieb_Hamiltonian(L, gamma, h=0.0, anisotropy_axis='z'):
    g = nk.graph.Hypercube(length=L, n_dim=1, pbc=True)
    hilbert = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
    H = nk.operator.LocalOperator(hilbert, dtype=complex)

    # Exchange couplings
    axes  = ['x', 'y', 'z']
    op_map = {
        'x': nk.operator.spin.sigmax,
        'y': nk.operator.spin.sigmay,
        'z': nk.operator.spin.sigmaz,
    }
    rotate = np.array([(-1)**i for i in range(L)])

    # Build the anisotropic exchange
    for i in range(L):
        j = (i + 1) % L
        for axis in axes:
            J = (1+gamma) if axis==anisotropy_axis else (1-gamma)
            op_i = op_map[axis](hilbert, i)
            op_j = op_map[axis](hilbert, j)
            factor = rotate[i]*rotate[j] if axis in ('x','y') else 1
            H += (J/4) * factor * (op_i @ op_j)

    # Add Zeeman bias -(h) sum_i S^z_i
    if abs(h) > 0:
        for i in range(L):
            H += -h * nk.operator.spin.sigmaz(hilbert, i)

    return H, g, hilbert

exact_energies = []
fidelities = []
pred_energies = []
pred_variances = []
gammas = []

for gamma in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
    # Parameters
    L = 20
    h_initial = gamma + 0.2       # start with a small bias
    n_warmup_stages = 15       # number of annealing blocks
    n_iter_per = 333      # VMC iterations per block

    # Build initial biased Hamiltonian and variational state
    H, g, hilbert = Lieb_Hamiltonian(L, gamma, h=h_initial)
    ma = sym_sinekan.SymmetricSineKAN1D(layers_hidden=(64,64,1), grid_size=8)
    vs = nk.vqs.MCState(
        nk.sampler.MetropolisLocal(hilbert, n_chains=1024),
        ma,
        n_samples=1024,
        chunk_size=1024,
        seed=42,
    )

    # Optimizer & scheduler (same throughout)
    n_warmup_epochs = n_warmup_stages * n_iter_per
    opt = nk.optimizer.Adam(learning_rate=1e-4)

    vmc = nk.VMC(H, optimizer=opt, variational_state=vs)

    # Adiabatic annealing loop
    current_epoch = 0
    for stage, h in enumerate(np.linspace(h_initial, 0.0, n_warmup_stages)):
        H, _, _ = Lieb_Hamiltonian(L, gamma, h)

        # update the existing VMC
        vmc = nk.VMC(H, optimizer=opt, variational_state=vs)

        # continue optimizing in the same variational_state
        vmc.run(
            n_iter=n_iter_per,
            out=f"AHM_SymSineKAN_stage{stage:02d}"
        )


    scheduler = optax.schedules.linear_schedule(1e-4, 1e-6, 10_000 - n_warmup_epochs, 0)
    opt = nk.optimizer.Adam(learning_rate=scheduler)

    H, _, _ = Lieb_Hamiltonian(L, gamma, h=0.0)

    # update the existing VMC
    vmc = nk.VMC(H, optimizer=opt, variational_state=vs)

    # continue optimizing in the same variational_state
    vmc.run(
        n_iter=10_000 - n_warmup_epochs,
        out=f"AHM_SymSineKAN_stage{stage:02d}"
    )


    psi_ed = nk.exact.lanczos_ed(H, compute_eigenvectors=True)[1][:,0]
    psi_nqs = vs.to_array(normalize=False)

    num = abs(np.vdot(psi_ed, psi_nqs))**2
    den = (np.vdot(psi_ed, psi_ed).real * np.vdot(psi_nqs, psi_nqs).real)
    fidelity = num/den

    lanczos_result = nk.exact.lanczos_ed(H, compute_eigenvectors=True)

    # Extract ground state energy and wavefunction
    ground_state_energy = lanczos_result[0][0]  # First element of the first tuple element (eigenvalues)
    psi_ed = lanczos_result[1][:, 0]  # Second element of the tuple (eigenvectors), first column

    print(f"Gamma: {gamma}")
    print(f"Exact Ground state energy: {ground_state_energy:.10f}")
    print(f"Ground state energy from VMC: {vs.expect(H).mean.real:.10f}")
    print(f"Ground state energy variance from VMC: {vs.expect(H).variance.real:.10f}")
    print(f"Fidelity between SymSineKAN and exact ground state: {fidelity:.10f}")
    gammas.append(gamma)
    exact_energies.append(ground_state_energy)
    fidelities.append(fidelity)
    pred_energies.append(vs.expect(H).mean.real)
    pred_variances.append(vs.expect(H).variance.real)

In [ ]:
# Lieb-Schultz-Matttis Hamiltonian
import netket as nk
import numpy as np
from netket.operator.spin import sigmax, sigmay, sigmaz
from netket.exact import lanczos_ed
import optax

def Lieb_Hamiltonian(L, gamma, h=0.0, anisotropy_axis='z'):
    g = nk.graph.Hypercube(length=L, n_dim=1, pbc=True)
    hilbert = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
    H = nk.operator.LocalOperator(hilbert, dtype=complex)

    # Exchange couplings
    axes  = ['x', 'y', 'z']
    op_map = {
        'x': nk.operator.spin.sigmax,
        'y': nk.operator.spin.sigmay,
        'z': nk.operator.spin.sigmaz,
    }
    rotate = np.array([(-1)**i for i in range(L)])

    # Build the anisotropic exchange
    for i in range(L):
        j = (i + 1) % L
        for axis in axes:
            J = (1+gamma) if axis==anisotropy_axis else (1-gamma)
            op_i = op_map[axis](hilbert, i)
            op_j = op_map[axis](hilbert, j)
            factor = rotate[i]*rotate[j] if axis in ('x','y') else 1
            H += (J/4) * factor * (op_i @ op_j)

    # Add Zeeman bias -(h) sum_i S^z_i
    if abs(h) > 0:
        for i in range(L):
            H += -h * nk.operator.spin.sigmax(hilbert, i)

    return H, g, hilbert

for gamma in [0.9, 1.0]:
    # Parameters
    L = 20
    h_initial = gamma + 0.2       # start with a small bias
    n_warmup_stages = 15       # number of annealing blocks
    n_iter_per = 333      # VMC iterations per block

    # Build initial biased Hamiltonian and variational state
    H, g, hilbert = Lieb_Hamiltonian(L, gamma, h=h_initial)
    ma = sym_sinekan.SymmetricSineKAN1D(layers_hidden=(64,64,1), grid_size=8)
    vs = nk.vqs.MCState(
        nk.sampler.MetropolisLocal(hilbert, n_chains=1024),
        ma,
        n_samples=1024,
        chunk_size=1024,
        seed=42,
    )

    # Optimizer & scheduler (same throughout)
    n_warmup_epochs = n_warmup_stages * n_iter_per
    opt = nk.optimizer.Adam(learning_rate=1e-4)

    vmc = nk.VMC(H, optimizer=opt, variational_state=vs)

    # Adiabatic annealing loop
    current_epoch = 0
    for stage, h in enumerate(np.linspace(h_initial, 0.0, n_warmup_stages)):
        H, _, _ = Lieb_Hamiltonian(L, gamma, h)

        # update the existing VMC
        vmc = nk.VMC(H, optimizer=opt, variational_state=vs)

        # continue optimizing in the same variational_state
        vmc.run(
            n_iter=n_iter_per,
            out=f"AHM_SymSineKAN_stage{stage:02d}"
        )


    scheduler = optax.schedules.linear_schedule(1e-4, 1e-6, 10_000 - n_warmup_epochs, 0)
    opt = nk.optimizer.Adam(learning_rate=scheduler)

    H, _, _ = Lieb_Hamiltonian(L, gamma, h=0.0)

    # update the existing VMC
    vmc = nk.VMC(H, optimizer=opt, variational_state=vs)

    # continue optimizing in the same variational_state
    vmc.run(
        n_iter=10_000 - n_warmup_epochs,
        out=f"AHM_SymSineKAN_stage{stage:02d}"
    )

    if gamma == 0.9:
        psi_ed = nk.exact.lanczos_ed(H, compute_eigenvectors=True)[1][:,0]
        psi_nqs = vs.to_array(normalize=False)

        num = abs(np.vdot(psi_ed, psi_nqs))**2
        den = (np.vdot(psi_ed, psi_ed).real * np.vdot(psi_nqs, psi_nqs).real)
        fidelity = num/den

        lanczos_result = nk.exact.lanczos_ed(H, compute_eigenvectors=True)

        # Extract ground state energy and wavefunction
        ground_state_energy = lanczos_result[0][0]  # First element of the first tuple element (eigenvalues)
    else:
        lanczos_result = nk.exact.lanczos_ed(H, k=2, compute_eigenvectors=True)

        psi_ed1 = lanczos_result[1][:,0]
        psi_ed2 = lanczos_result[1][:,1]
        psi_nqs = vs.to_array(normalize=False)

        num1 = abs(np.vdot(psi_ed1, psi_nqs))**2
        den1 = (np.vdot(psi_ed1, psi_ed1).real * np.vdot(psi_nqs, psi_nqs).real)
        fidelity1 = num1/den1
        num2 = abs(np.vdot(psi_ed2, psi_nqs))**2
        den2 = (np.vdot(psi_ed2, psi_ed2).real * np.vdot(psi_nqs, psi_nqs).real)
        fidelity2 = num2/den2
        fidelity = fidelity1 + fidelity2


        # Extract ground state energy and wavefunction
        ground_state_energy = lanczos_result[0][0]  # First element of the first tuple element (eigenvalues)

    print(f"Gamma: {gamma}")
    print(f"Exact Ground state energy: {ground_state_energy:.10f}")
    print(f"Ground state energy from VMC: {vs.expect(H).mean.real:.10f}")
    print(f"Ground state energy variance from VMC: {vs.expect(H).variance.real:.10f}")
    print(f"Fidelity between SymSineKAN and exact ground state: {fidelity:.10f}")
    gammas.append(gamma)
    exact_energies.append(ground_state_energy)
    fidelities.append(fidelity)
    pred_energies.append(vs.expect(H).mean.real)
    pred_variances.append(vs.expect(H).variance.real)

In [ ]:
print("Gammas")
for data in gammas:
    print(data)

print("Exact Energies")
for data in exact_energies:
    print(data)

print("Fidelities")
for data in fidelities:
    print(data)

print("Pred Energies")
for data in pred_energies:
    print(data)

print("Pred Variances")
for data in pred_variances:
    print(data)